## Meta matching v1.1
This jupyter notebook demonstrates you how to load and use meta-matching algorthm. In this demonstration, we performed Meta-matching (DNN) stacking with 100 example subjects.

Package needed (and version this jupyter notebook tested):
* Numpy (1.24.4)
* Scipy (1.10.1)
* PyTorch (2.0.0)
* Scikit-learn (1.3.2)

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
path_repo = '../'#'/home/the/deepGround/code/2002/Meta_matching_models/'

In [2]:
# initialization and random seed set

import os
import sys
import random
import scipy
import torch
import sklearn
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

import warnings
warnings.filterwarnings("ignore")

Please modify the gpu number here if you want to use different gpu. If the gpu you assigned not availiable, it will assign to cpu

In [3]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1. load data
Load the example data that we provided, it contains 
* Example input functional connectivity (FC) `x` with size of (100, 87571)
    * 100 is number of subjects
    * 87571 is flatten vector of 419 by 419 FC (419*418/2=87571)
    * We perform participant-wise normalization (demean the FC vector and devide by l2 norm of the FC vector for each participant), using mean and std of 87571 elements of each subject
* Example output phenotypes `y` with size of (100, 3)
    * 3 is number of phenotypes.

In [4]:
path_v11 = os.path.join(path_repo, 'v1.1')
path_data = os.path.join(path_repo, 'data')
sys.path.append(os.path.join(path_repo, "utils"))
from CBIG_model_pytorch import demean_norm

npz = os.path.join(path_data, 'meta_matching_example_data.npz')
npz = np.load(npz)
print(npz.files)
x_input = npz['x']
y_input = npz['y']
x_input = demean_norm(x_input)
print(x_input.shape, y_input.shape)

['x', 'y']
(100, 87571) (100, 3)


### Step 2. Split data
Here, we also split 100 subjects to 80/20, where 80 for training, and 20 for test.

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(80, 87571) (20, 87571) (80, 3) (20, 3)


After we split out the subjects, we apply z-normalization to both training and testing set, using data in the training set.

In [7]:
def z_norm(y_train, y_test):
    # subtract mean of y of the training set
    t_mu = np.nanmean(y_train, axis=0, keepdims=True)
    y_train = y_train - t_mu
    y_test = y_test - t_mu

    # divide std of y of the training set
    t_sigma = np.nanstd(y_train, axis=0)
    if y_train.ndim == 2:
        t_sigma_d = t_sigma[np.newaxis, :]
    else:
        t_sigma_d = t_sigma
        if t_sigma == 0:
            return y_train, y_test
    y_train = y_train / t_sigma_d
    y_test = y_test / t_sigma_d
    return y_train, y_test

y_train, y_test = z_norm(y_train, y_test)

### Step 3. Prepare data for PyTorch model
Then we prepare data for DNN model, we will input both the `x_train` and `x_test` into the model to get the predicted phenotypes. 

For meta-matching (DNN) stacking, we do not need real phenotype for the DNN model, I created all zeros `y_dummy` just for function requirement. In some other cases, like meta-matching (DNN) finetuning, you need to use real phenotype data here.

In [8]:
from torch.utils.data import DataLoader
from CBIG_model_pytorch import multi_task_dataset

batch_size = 16
y_dummy = np.zeros(y_train.shape)
dset_train = multi_task_dataset(x_train, y_dummy, True)
trainLoader = DataLoader(dset_train,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=1)

y_dummy = np.zeros(y_test.shape)
dset_test = multi_task_dataset(x_test, y_dummy, True)
testLoader = DataLoader(dset_test,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=1)

### Step 4. load model
Here we load the meta-matching model saved, it is a DNN that takes FC as input and output 67 phenotypes prediction trained on 67 UK Biobank phenotypes

In [9]:
path_model_weight = os.path.join(path_v11, 'meta_matching_v1.1_model.pkl_torch') 
net = torch.load(path_model_weight, map_location=device)
net.to(device)
net.train(False)
print(net)

dnn(
  (layers): ModuleList(
    (0): Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=87571, out_features=256, bias=True)
      (2): ReLU()
      (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1-2): 2 x Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=256, out_features=256, bias=True)
      (2): ReLU()
      (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=256, out_features=67, bias=True)
    )
  )
)


### Step 5. DNN model predict
Here we apply the DNN trained on 67 UK Biobank phenotypes to predict the 67 phenotypes on `x_train` and `x_test`. We will get the predicted 67 phenotypes on both 80 training subjects and 20 test subjects.

In [10]:
y_train_pred = np.zeros((0, 67))
for (x, _) in trainLoader:
    x= x.to(device)
    outputs = net(x)
    y_train_pred = np.concatenate((y_train_pred, outputs.data.cpu().numpy()), axis=0)
print(y_train_pred.shape, '\n', y_train_pred)

y_test_pred = np.zeros((0, 67))
for (x, _) in testLoader:
    x= x.to(device)
    outputs = net(x)
    y_test_pred = np.concatenate((y_test_pred, outputs.data.cpu().numpy()), axis=0)
print(y_test_pred.shape, '\n', y_test_pred)

(80, 67) 
 [[-0.15115939 -0.0669269  -0.54655945 ... -0.11759652 -0.50293493
  -0.37641898]
 [ 0.29037341 -0.20854312  0.23705281 ...  0.15359323 -0.07533579
  -0.01452626]
 [ 0.10086952  0.20367797 -0.53936934 ... -0.04685985 -0.14448746
  -0.0352214 ]
 ...
 [ 0.05406483 -0.03227098  0.01010649 ...  0.10382769  0.06453447
   0.05130197]
 [-0.02054616 -0.17987496  0.16705912 ...  0.1099993  -0.22622883
  -0.34170136]
 [-0.10911851  0.1235078  -0.43411684 ... -0.07488919 -0.53739721
  -0.25947312]]
(20, 67) 
 [[-0.05762977 -0.17934641  0.16057274 ...  0.16511196 -0.55663419
  -0.25849211]
 [ 0.01560813 -0.1447289   0.19979201 ...  0.13954791  0.01221326
  -0.41223368]
 [ 0.05661141 -0.21707436  0.19259959 ...  0.1661057  -0.24444117
  -0.16643339]
 ...
 [-0.01382108 -0.12719817  0.20016837 ...  0.12121779 -0.3225328
   0.05766377]
 [-0.15039191 -0.21929874  0.01748165 ...  0.12292579 -0.37921736
  -0.48630229]
 [ 0.15801945  0.05928478 -0.45958135 ... -0.1126053   0.15486319
  -0.214501

### Step 6. Stacking
Perform stacking with `y_train_pred`, `y_test_pred`, `y_train`, where we use the prediction of 80 subjects `y_train_pred` (input) and real data `y_train` (output) to train the stacking model (you can either use all 67 source phenotypes for stacking, or select top K source phenotypes relevant to the target phenotype, like we mentioned in our paper; it turns out that these 2 ways achieves similar performances), then we applied the model to `y_test_pred` to get final prediction of 3 phenotypes on 20 subjects.

#### Hyperparameter Tuning 
In `stacking()` function, we set the range of `alpha` as `[0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20]`. You are weclomed to modify the range of `alpha` to get better performance on your own data.

In [11]:
from CBIG_model_pytorch import stacking
y_test_final=np.zeros((y_test_pred.shape[0], y_train.shape[1]))
for i in range(y_train.shape[1]):
    # For each test phenotype, perform stacking by developing a KRR model
    y_test_temp, _ = stacking(y_train_pred, y_test_pred, y_train[:,i].view(), [0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20])
    y_test_final[:,i] = y_test_temp.flatten()
print(y_test_final.shape, '\n', y_test_final)

(20, 3) 
 [[ 0.4913766   0.62631824  0.18645581]
 [ 0.38043247  0.34795771  0.25650832]
 [ 0.47721059  0.50117837  0.1833064 ]
 [-0.3285022  -0.27597382 -0.12594698]
 [ 0.34427317  0.26884023  0.21047081]
 [-0.14829715 -0.14942112 -0.09516254]
 [ 0.29252215  0.29282001  0.20900234]
 [-0.39725529 -0.36118301 -0.19106383]
 [-0.41640963 -0.57297535 -0.09578888]
 [ 0.34701785  0.28048247  0.21947869]
 [-0.48536224 -0.65088255 -0.1103963 ]
 [ 0.14344771  0.2514617   0.07791609]
 [ 0.40045764  0.36931898  0.2105459 ]
 [ 0.23353501  0.15623559  0.17580163]
 [ 0.2169887   0.20669226  0.23414636]
 [ 0.15605415  0.26773961 -0.02709453]
 [-0.40748142 -0.34518476 -0.0845362 ]
 [ 0.24727978  0.25942492 -0.09948186]
 [ 0.44207504  0.48973673  0.30049635]
 [-0.33514239 -0.46463813 -0.02771175]]


### Step 7. Evaluation
Evaluate the prediction performance. Note that we didn't reverse the z-normalization which previously applied on `y_train` and `y_test`. This is because the metric (Pearson correlation) would not be affected by z-normalization. If you are predicting the phenotypes for practical use, you are recommended to reverse the z-normalization

In [12]:
from scipy.stats.stats import pearsonr
corr = np.zeros((y_train.shape[1]))
for i in range(y_train.shape[1]):
    corr[i] = pearsonr(y_test_final[:, i], y_test[:, i])[0]
print(corr)

[0.35514837 0.3138165  0.33030006]


### Step 8. Haufe transform predictive network features (PNFs) computation
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each element of FC on the 80 training subjects. The final PNF is in shape of (87571, 3), where 87571 is number of 419 by 419 FC elements, and 3 is number of phenotypes.

In [13]:
from CBIG_model_pytorch import covariance_rowwise

y_train_haufe, _ = stacking(y_train_pred, y_train_pred, y_train)
print(y_train_haufe.shape)
cov = covariance_rowwise(x_train, y_train_haufe)
print(cov, '\n', cov.shape)

(80, 3)
[[-1.69506336e-04 -1.54628114e-04 -1.11766445e-04]
 [-6.55518003e-05 -5.72385588e-05 -5.66451097e-05]
 [-2.30731702e-04 -2.20008981e-04 -2.12088125e-04]
 ...
 [ 1.44474389e-04  1.24612995e-04  1.43483677e-04]
 [ 2.06369212e-04  1.83499597e-04  1.78814555e-04]
 [ 1.04217654e-04  8.84985452e-05  4.54989622e-05]] 
 (87571, 3)


### Step 9. Haufe transform predictive network features (PNFs) computation for training phenotypes
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each training phenotypes on the 80 training subjects. The final PNF is in shape of (67, 3), where 67 is number of training phenotypes, and 3 is number of phenotypes.

In [14]:
from CBIG_model_pytorch import covariance_rowwise

cov = covariance_rowwise(y_train_pred, y_train_haufe)
print(cov, '\n', cov.shape)

[[ 1.32608351e-02  9.63462855e-03  6.54047545e-03]
 [-3.47620989e-02 -3.51622313e-02 -1.23244510e-02]
 [ 1.02205852e-01  9.95429640e-02  4.72639905e-02]
 [ 1.18318519e-03  3.37838055e-03  3.06332652e-03]
 [ 8.04337794e-03  5.31870769e-03 -6.85581514e-03]
 [-1.16693944e-02 -1.35825513e-02 -2.67586990e-02]
 [ 1.35861038e-02  1.41768973e-02  1.63911172e-02]
 [ 1.64290300e-02  1.66009107e-02  2.22180825e-02]
 [ 7.13119308e-02  6.46184689e-02  3.56405776e-02]
 [ 1.69930153e-04 -4.19885311e-03 -7.71350143e-03]
 [ 4.54026476e-02  4.00091577e-02  3.43061995e-02]
 [-4.44247298e-02 -4.71443056e-02 -1.91862230e-02]
 [-1.50066688e-02 -1.32138653e-02 -1.26282887e-02]
 [ 6.42879988e-02  6.00446199e-02  6.79775498e-02]
 [ 3.01064308e-02  2.72376669e-02  2.87876017e-02]
 [ 2.61667808e-01  2.61756351e-01  1.33788236e-01]
 [-5.40860783e-02 -4.52216158e-02 -3.98557722e-02]
 [-1.95923281e-02 -5.36350048e-03 -3.71716724e-02]
 [-6.76770744e-02 -6.14991707e-02 -5.84364257e-02]
 [-9.53418702e-03 -8.99053206e-